<a href="https://colab.research.google.com/github/GGpark1/User-Log-data-analytics/blob/main/Collaborative_filltering_elec_without_cherry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 구매 기록이 한번도 없는 상품은 추천 목록에서 제외 -> 희소문제를 해결하기 위함 
  - 전자 제품 군의 14,950개의 상품 중에서 한번도 팔리지 않은 상품은 약 10,000개로 분석에 포함될 시 희소문제를 일으킬 가능성이 높음

- 편향 보정을 위해 스마트폰을 구매한 일부 비율을 뺄까?

## 구매 횟수 필터 없는 추천 시스템

In [3]:
from pandas.io import gbq

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#pd.options.display.float_format = "{:,.2f}".format
import warnings
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
warnings.simplefilter("ignore")

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [4]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data_without_cherry
WHERE category_code LIKE 'electronics%' AND event_type = 'purchase'
GROUP BY user_id, product_id
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=lnHgJjQALQ4myTApqbzlWX5GC7rNIC&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWiA0tviwIVl-6qWBc9NfINgKf7gQmV-fh8JJR2Olt3NZ2ODVytW4xk


In [5]:
df_ratings = df

In [13]:
base = df_ratings['purchase_count'].mean()

In [6]:
sql = """SELECT product_id, brand, category_code FROM `empirical-weft-350709.CP2_ggpark.log_data`
WHERE event_type = 'purchase' AND category_code LIKE 'electronics%'
GROUP BY product_id, brand, category_code"""
project_id = 'empirical-weft-350709'

df_product = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [7]:
product_data = df_product
product_data.head()

,product_id,brand,category_code
0,11300053,texet,electronics.telephone
1,22400122,trust,electronics.audio.microphone
2,11300005,panasonic,electronics.telephone
3,9900453,byintek,electronics.video.projector
4,18500081,huion,electronics.tablet


In [10]:
df_fin = pd.pivot_table(df, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')

In [11]:
df_fin

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400220,22400236,51600003,51600004,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
303160429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340041246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401021311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403013066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410333972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566263549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566265448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566270060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_base_line = df_fin.fillna(base)

In [15]:
df_final = df_fin.fillna(0)

In [16]:
df_final['total'] = df_final.sum(axis=1)
df_filtered = df_final.loc[df_final['total'] > 1]
df_filtered.drop(columns='total', inplace = True)

df_filtered

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400220,22400236,51600003,51600004,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415873351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
427391662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
437371552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446080337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566240992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566243643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566249028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
matrix = df_filtered.values
user_ratings_mean = np.mean(matrix, axis = 1)

In [18]:
user_ratings_mean_reshape = user_ratings_mean.reshape(-1, 1)

In [19]:
matrix_user_mean = matrix - user_ratings_mean_reshape

In [20]:
pd.DataFrame(matrix_user_mean, columns = df_filtered.columns, index = df_filtered.index).head()

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400220,22400236,51600003,51600004,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,...,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045
415873351,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,...,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045
427391662,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,...,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045
437371552,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,...,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045
446080337,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,...,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045,-0.00045


In [22]:
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [23]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(72017, 12)
(12,)
(12, 4448)


In [24]:
sigma = np.diag(sigma)

In [25]:
sigma.shape

(12, 12)

In [26]:
sigma[0]

array([137.43243485,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [27]:
dot_matrix = np.dot(U, sigma)

In [28]:
df_svd_predicted_purchase = np.dot(dot_matrix, Vt)

In [29]:
df_svd_predicted_purchase = df_svd_predicted_purchase + user_ratings_mean_reshape

In [30]:
df_svd_preds = pd.DataFrame(df_svd_predicted_purchase, columns = df_filtered.columns, index = df_filtered.index)
df_svd_preds.head()

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400220,22400236,51600003,51600004,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,-0.000099,-0.000037,-0.000042,0.000239,0.000435,0.000422,0.000879,0.000008,0.000314,-0.000037,...,-0.000037,-0.000037,-0.000038,-0.000037,-0.000037,-0.000037,-0.000037,-0.000037,-0.000037,-0.000038
415873351,0.000436,0.000409,0.000408,0.000417,0.000427,0.000487,0.000426,0.000485,0.000502,0.000406,...,0.000406,0.000406,0.000406,0.000406,0.000406,0.000406,0.000406,0.000406,0.000406,0.000406
427391662,0.000008,0.000108,0.000096,0.000104,0.000087,-0.000097,0.000046,0.000342,0.000065,0.000100,...,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100
437371552,0.000426,0.000420,0.000419,0.000431,0.000438,0.000461,0.000455,0.000444,0.000456,0.000419,...,0.000419,0.000419,0.000419,0.000419,0.000419,0.000419,0.000419,0.000419,0.000419,0.000419
446080337,0.000070,0.000090,0.000094,0.000088,0.000261,0.000726,0.000238,0.000350,0.000261,0.000096,...,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096


In [28]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_movies_df, on = 'product_id').sort_values(['purchase_count'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['product_id'].isin(user_history['product_id'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'product_id')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [32]:
already_rated, predictions = recommend_movies(df_svd_preds, 523974502, product_data, df_ratings, 10)

In [33]:
predictions

,product_id,brand,category_code,Predictions
2124,5100866,samsung,electronics.clocks,0.139451
1458,5100860,samsung,electronics.clocks,0.115699
1334,1005031,xiaomi,electronics.smartphone,0.085351
1466,1801739,samsung,electronics.video.tv,0.078504
1315,1004739,xiaomi,electronics.smartphone,0.058427
1452,5100577,samsung,electronics.clocks,0.049626
1450,5100564,samsung,electronics.clocks,0.032579
1195,5100722,huawei,electronics.clocks,0.026911
1524,1004836,samsung,electronics.smartphone,0.026871
1454,5100719,samsung,electronics.clocks,0.026589


In [32]:
def evaluate(sparse_matrix, pred_matrix):
        idx, jdx = sparse_matrix.to_numpy().nonzero()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(sparse_matrix.iloc[i, j])
            preds.append(pred_matrix.iloc[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [36]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_base_line)

2.1780562815743534

In [37]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_svd_preds)

2.0735755409607584

## 구매 횟수 필터 있는 추천 시스템

In [8]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data_without_cherry
WHERE category_code LIKE 'electronics%' AND event_type = 'purchase'
GROUP BY user_id, product_id
HAVING purchase_count <= 2
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df_limit = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [9]:
df_limit.sort_values('purchase_count')

,user_id,product_id,purchase_count
126587,549170918,1005004,1
168776,545012255,1004749,1
168777,551307859,1004655,1
168778,564283981,1004767,1
168779,564288654,1004657,1
...,...,...,...
23965,551763104,1005166,2
23964,518810868,1004741,2
23963,559519141,1005161,2
23969,525450902,1005160,2


In [10]:
df_ratings_limit = df_limit
mean = df_ratings_limit['purchase_count'].mean()

In [11]:
df_fin_limit = pd.pivot_table(df_limit, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')
df_fin_limit

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400220,22400236,51600003,51600004,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
303160429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340041246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401021311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403013066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410333972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566263549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566265448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566270060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_final_limit = df_fin_limit.fillna(0)

In [13]:
df_base_line_limit = df_fin_limit.fillna(mean)

In [14]:
df_final_limit['total'] = df_final_limit.sum(axis=1)
df_filtered_limit = df_final_limit.loc[df_final_limit['total'] > 1]
df_filtered_limit.drop(columns='total', inplace = True)

df_filtered_limit

product_id,1000978,1001588,1002042,1002062,1002098,1002099,1002100,1002101,1002102,1002103,...,22400220,22400236,51600003,51600004,51600009,51600010,51600011,51600012,51600018,51600020
user_id,,,,,,,,,,,,,,,,,,,,,
403013066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415873351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
427391662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
437371552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446080337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566240992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566243643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566249028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
matrix_limit = df_filtered_limit.values
user_ratings_mean_limit = np.mean(matrix_limit, axis = 1)

In [16]:
user_ratings_mean_reshape_limit = user_ratings_mean_limit.reshape(-1, 1)

In [17]:
matrix_user_mean_limit = matrix_limit - user_ratings_mean_reshape_limit

In [20]:
U_limit, sigma_limit, Vt_limit = svds(matrix_user_mean_limit, k = 12)

In [22]:
sigma_limit = np.diag(sigma_limit)

In [23]:
dot_matrix_limit = np.dot(U_limit, sigma_limit)

In [24]:
df_svd_predicted_purchase_limit = np.dot(dot_matrix_limit, Vt_limit)

In [25]:
df_svd_predicted_purchase_limit = df_svd_predicted_purchase_limit + user_ratings_mean_reshape_limit

In [ ]:
df_svd_preds_limit = pd.DataFrame(df_svd_predicted_purchase_limit, columns = df_filtered_limit.columns, index = df_filtered_limit.index)
df_svd_preds_limit.head()

In [29]:
already_rated, predictions = recommend_movies(df_svd_preds_limit, 562094335, product_data, df_ratings_limit, 10)

In [30]:
predictions

,product_id,brand,category_code,Predictions
1528,1004836,samsung,electronics.smartphone,0.051518
1523,1004833,samsung,electronics.smartphone,0.042280
1525,1004856,samsung,electronics.smartphone,0.028744
1542,1004873,samsung,electronics.smartphone,0.027694
973,1002524,apple,electronics.smartphone,0.022239
1522,1004750,samsung,electronics.smartphone,0.012773
1527,1004870,samsung,electronics.smartphone,0.011679
968,1002633,apple,electronics.smartphone,0.010777
1526,1004767,samsung,electronics.smartphone,0.005743
1303,1004741,xiaomi,electronics.smartphone,0.005291


In [33]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_base_line_limit)

0.44927815038680446

In [34]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_svd_preds_limit)

1.0663149085092478